In [51]:
import numpy as np
from random import random

from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import QControlBox
from pytket.circuit import CircBox


In [141]:
# m is precision of answer
def iter_circ(m_bits, eigenstate: Circuit, unitary: Circuit) -> Circuit:  
    # create base of circuit to return
    iter_circ: Circuit = Circuit(1, m_bits + 1)
    qreg = iter_circ.get_q_register("q")
    creg = iter_circ.get_c_register("c")
    ancil = qreg[0]
    n_state_prep_qubits = eigenstate.n_qubits
    sreg = iter_circ.add_q_register("psi", n_state_prep_qubits)
    iter_circ.add_circuit(eigenstate, list(sreg))
    # add hadamard to ancillary qubit
    iter_circ.H(0)
    # Create a controlled unitary with a single control qubit
    unitary.name = "U"
    controlled_u_gate = QControlBox(CircBox(unitary), 1)
    
    for j in range(m_bits, 0, -1):
        # add unitaries
        for _ in range(2 ** (j - 1)):
            iter_circ.add_qcontrolbox(controlled_u_gate, [ancil] + list(sreg))
    
        for expn in range(m_bits + 1 - j, 1, -1):
            phase = - 2 * np.pi / (2 ** expn)
            iter_circ.U1(phase, 0, condition = creg[expn + j - 1])
            print(expn, expn + j - 1)
        
        iter_circ.H(0)
    
        iter_circ.Measure(ancil, creg[j])
    
    # Add the initial state
    return iter_circ
    # # Create the unitary:
    # unitary.name = "U"
    # cu_gate = QControlBox(CircBox(unitary), 1)
    # render_circuit_jupyter(iter_circ)
    # pass

## Call the function, run experiments and stuff

In [143]:
#input_angle = 0.5 
# From 0 to 1
input_angle = 0.73
initial_state = Circuit(1).X(0) #Make sure to just add quantum channels here
unitary = Circuit(1).U1(input_angle, 0)
circuit_iter_to_render = iter_circ(4, 
    eigenstate=initial_state, unitary=unitary
)

render_circuit_jupyter(circuit_iter_to_render)

2 4
3 4
2 3
4 4
3 3
2 2


In [144]:
from pytket.extensions.nexus import NexusBackend, QuantinuumConfig, Nexus
from datetime import datetime

In [147]:
phase_est_project = Nexus().new_project(f"IPE_test2 - {datetime.now()}")

configuration = QuantinuumConfig(device_name="H1-1LE", user_group="iQuHACK_2024")
backend = NexusBackend(
    backend_config= configuration, 
    project= phase_est_project
)


New project created: IPE_test2 - 2024-02-04 06:49:59.810419

Started using project with name: IPE_test2 - 2024-02-04 06:49:59.810419


In [148]:
compiled_circ = backend.get_compiled_circuit(circuit_iter_to_render)

RetryError: HTTPSConnectionPool(host='nexus.quantinuum.com', port=443): Max retries exceeded with url: /api/v5/experiments/20fec34f-f840-49f8-b135-20219f3f9aa2/circuits (Caused by ResponseError('too many 500 error responses'))